# Exercise 4: Creating Custom Tools
## Building Financial Calculator and Data Lookup Tools

In this notebook, we'll learn how to create custom tools that can be used by LangChain agents. We'll build practical financial tools including a compound interest calculator and a stock price lookup tool.

## Learning Objectives
- Understand the Tool class and how to create custom tools
- Learn function wrapping techniques for tool creation
- Write effective tool descriptions for agent use
- Build financial calculation tools
- Create data lookup tools with mock data
- Integrate tools with LangChain agents

Let's build some powerful financial tools! 🛠️📊

## Setup and Imports

First, let's install and import the necessary libraries for creating custom tools.

In [28]:
# Install required packages
!uv add langchain langchain-openai langchain-community python-dotenv

Resolved 392 packages in 0.84ms
Audited 223 packages in 0.09ms


In [29]:
# Import necessary libraries
from langchain.tools import Tool, StructuredTool, tool
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
import os
from dotenv import load_dotenv
import json
import random
from datetime import datetime, timedelta
from typing import Optional, Dict, Any
import math

# Load environment variables
load_dotenv()

# Initialize ChatOpenAI
llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.1,  # Lower temperature for more consistent calculations
    api_key=os.getenv("OPENAI_API_KEY")
)

print("✓ Setup complete!")
print("✓ Ready to build custom tools!")

✓ Setup complete!
✓ Ready to build custom tools!


## Part 1: Understanding Tool Creation

Let's start by understanding the basic structure of a LangChain tool and create our first simple tool.

In [30]:
# Simple example: Basic calculator tool
def simple_calculator(operation: str) -> str:
    """
    Performs basic mathematical operations.
    
    Args:
        operation: A string like "2+3" or "10*5"
    
    Returns:
        The result of the calculation
    """
    try:
        # Simple eval for demo purposes (not recommended for production)
        result = eval(operation)
        return f"The result of {operation} is {result}"
    except Exception as e:
        return f"Error in calculation: {str(e)}"

# Create a tool from the function
calculator_tool = Tool(
    name="Calculator",
    description="Use this tool to perform basic mathematical operations. Input should be a mathematical expression like '2+3' or '10*5'.",
    func=simple_calculator
)

# Test the tool
print("Testing Calculator Tool:")
print(calculator_tool.run("25 * 4"))
print(calculator_tool.run("100 / 4"))
print("✓ Basic calculator tool created!")

# Alternative approach: Using the @tool decorator
@tool
def percentage_calculator(input_string: str) -> str:
    """
    Calculate percentage of a given amount.
    
    Args:
        input_string: Input in format "amount,percentage" (e.g., "10000,15" for 15% of $10,000)
    
    Returns:
        The percentage calculation result
    """
    try:
        # Parse the input string
        parts = input_string.split(',')
        if len(parts) != 2:
            return "Error: Input should be in format 'amount,percentage' (e.g., '10000,15')"
        
        amount = float(parts[0].strip())
        percentage = float(parts[1].strip())
        result = amount * (percentage / 100)
        return f"{percentage}% of ${amount:,.2f} is ${result:,.2f}"
    except ValueError:
        return "Error: Please provide valid numbers in format 'amount,percentage'"
    except Exception as e:
        return f"Error in percentage calculation: {str(e)}"

# Test the decorator-created tool
print("\nTesting Decorator-Created Tool:")
print(percentage_calculator.run("10000,15"))
print(percentage_calculator.run("2500,8.5"))
print("✓ Decorator-based tool created!")

print("\n🎯 Tool Creation Methods:")
print("1. Tool() class with function - flexible, explicit")
print("2. @tool decorator - clean, concise, automatic schema generation")

Testing Calculator Tool:
The result of 25 * 4 is 100
The result of 100 / 4 is 25.0
✓ Basic calculator tool created!

Testing Decorator-Created Tool:
15.0% of $10,000.00 is $1,500.00
8.5% of $2,500.00 is $212.50
✓ Decorator-based tool created!

🎯 Tool Creation Methods:
1. Tool() class with function - flexible, explicit
2. @tool decorator - clean, concise, automatic schema generation


## Part 2: Advanced Tool Creation with Structured Inputs

Now let's create more sophisticated tools with structured inputs using Pydantic models. We'll start with a compound interest calculator - a key financial calculation.

In [31]:
# Define input schema for compound interest calculator
class CompoundInterestInput(BaseModel):
    """Input schema for compound interest calculator."""
    principal: float = Field(description="Initial investment amount in dollars")
    annual_rate: float = Field(description="Annual interest rate as a decimal (e.g., 0.07 for 7%)")
    years: int = Field(description="Number of years to compound")
    compounds_per_year: int = Field(default=12, description="Number of times interest compounds per year (default: 12 for monthly)")

def compound_interest_calculator(
    principal: float,
    annual_rate: float,
    years: int,
    compounds_per_year: int = 12
) -> str:
    """
    Calculate compound interest using the formula: A = P(1 + r/n)^(nt)
    
    Args:
        principal: Initial investment amount
        annual_rate: Annual interest rate (as decimal, e.g., 0.07 for 7%)
        years: Number of years to compound
        compounds_per_year: Number of times interest compounds per year
    
    Returns:
        Detailed compound interest calculation results
    """
    try:
        # Calculate compound interest
        amount = principal * (1 + annual_rate / compounds_per_year) ** (compounds_per_year * years)
        interest_earned = amount - principal
        
        # Calculate some additional useful metrics
        effective_annual_rate = (1 + annual_rate / compounds_per_year) ** compounds_per_year - 1
        monthly_growth = (amount / principal) ** (1 / (years * 12)) - 1
        
        result = {
            "principal": principal,
            "annual_rate_percent": annual_rate * 100,
            "years": years,
            "compounds_per_year": compounds_per_year,
            "final_amount": round(amount, 2),
            "interest_earned": round(interest_earned, 2),
            "effective_annual_rate_percent": round(effective_annual_rate * 100, 2),
            "total_return_percent": round((interest_earned / principal) * 100, 2),
            "average_monthly_growth_percent": round(monthly_growth * 100, 4)
        }
        
        # Format the response
        response = f"""
📊 COMPOUND INTEREST CALCULATION RESULTS:

Initial Investment: ${result['principal']:,.2f}
Annual Interest Rate: {result['annual_rate_percent']}%
Investment Period: {result['years']} years
Compounding Frequency: {result['compounds_per_year']} times per year

💰 FINAL RESULTS:
Final Amount: ${result['final_amount']:,.2f}
Interest Earned: ${result['interest_earned']:,.2f}
Total Return: {result['total_return_percent']}%

📈 ADDITIONAL METRICS:
Effective Annual Rate: {result['effective_annual_rate_percent']}%
Average Monthly Growth: {result['average_monthly_growth_percent']}%

🎯 SUMMARY: Your ${result['principal']:,.2f} investment will grow to ${result['final_amount']:,.2f} over {result['years']} years, earning ${result['interest_earned']:,.2f} in compound interest!
        """
        
        return response.strip()
        
    except Exception as e:
        return f"Error calculating compound interest: {str(e)}"

# Create the structured tool
compound_interest_tool = StructuredTool.from_function(
    func=compound_interest_calculator,
    name="CompoundInterestCalculator",
    description="Calculate compound interest for investments. Use this tool to determine how much an investment will grow over time with compound interest. Provide principal amount, annual interest rate (as decimal), number of years, and compounding frequency.",
    args_schema=CompoundInterestInput
)

# Test the tool
print("Testing Compound Interest Calculator:")
test_result = compound_interest_tool.run({
    "principal": 10000,
    "annual_rate": 0.07,
    "years": 10,
    "compounds_per_year": 12
})
print(test_result)

Testing Compound Interest Calculator:
📊 COMPOUND INTEREST CALCULATION RESULTS:

Initial Investment: $10,000.00
Annual Interest Rate: 7.000000000000001%
Investment Period: 10 years
Compounding Frequency: 12 times per year

💰 FINAL RESULTS:
Final Amount: $20,096.61
Interest Earned: $10,096.61
Total Return: 100.97%

📈 ADDITIONAL METRICS:
Effective Annual Rate: 7.23%
Average Monthly Growth: 0.5833%

🎯 SUMMARY: Your $10,000.00 investment will grow to $20,096.61 over 10 years, earning $10,096.61 in compound interest!


## Part 3: Stock Price Lookup Tool

Let's create a stock price lookup tool using mock data. In a real application, this would connect to a financial API.

In [32]:
# Create mock stock data
MOCK_STOCK_DATA = {
    "AAPL": {
        "name": "Apple Inc.",
        "sector": "Technology",
        "current_price": 185.92,
        "change": 2.34,
        "change_percent": 1.28,
        "volume": 52840000,
        "market_cap": 2.89e12,
        "pe_ratio": 28.5,
        "dividend_yield": 0.44,
        "52_week_high": 199.62,
        "52_week_low": 164.08
    },
    "GOOGL": {
        "name": "Alphabet Inc.",
        "sector": "Technology",
        "current_price": 142.56,
        "change": -1.23,
        "change_percent": -0.85,
        "volume": 28450000,
        "market_cap": 1.78e12,
        "pe_ratio": 25.2,
        "dividend_yield": 0.0,
        "52_week_high": 151.55,
        "52_week_low": 121.46
    },
    "MSFT": {
        "name": "Microsoft Corporation",
        "sector": "Technology",
        "current_price": 378.85,
        "change": 5.67,
        "change_percent": 1.52,
        "volume": 31250000,
        "market_cap": 2.81e12,
        "pe_ratio": 32.1,
        "dividend_yield": 0.73,
        "52_week_high": 384.30,
        "52_week_low": 309.45
    },
    "TSLA": {
        "name": "Tesla Inc.",
        "sector": "Automotive",
        "current_price": 248.50,
        "change": -8.23,
        "change_percent": -3.21,
        "volume": 98650000,
        "market_cap": 7.89e11,
        "pe_ratio": 58.7,
        "dividend_yield": 0.0,
        "52_week_high": 299.29,
        "52_week_low": 152.37
    },
    "AMZN": {
        "name": "Amazon.com Inc.",
        "sector": "Consumer Discretionary",
        "current_price": 155.89,
        "change": 3.45,
        "change_percent": 2.26,
        "volume": 42150000,
        "market_cap": 1.62e12,
        "pe_ratio": 43.8,
        "dividend_yield": 0.0,
        "52_week_high": 170.26,
        "52_week_low": 118.35
    }
}

# Input schema for stock lookup
class StockLookupInput(BaseModel):
    """Input schema for stock price lookup."""
    symbol: str = Field(description="Stock ticker symbol (e.g., AAPL, GOOGL, MSFT)")

def stock_price_lookup(symbol: str) -> str:
    """
    Look up current stock price and key financial metrics.
    
    Args:
        symbol: Stock ticker symbol (e.g., AAPL, GOOGL, MSFT)
    
    Returns:
        Current stock information including price, change, and key metrics
    """
    try:
        symbol = symbol.upper().strip()
        
        if symbol not in MOCK_STOCK_DATA:
            available_symbols = ", ".join(MOCK_STOCK_DATA.keys())
            return f"❌ Symbol '{symbol}' not found. Available symbols: {available_symbols}"
        
        stock = MOCK_STOCK_DATA[symbol]
        
        # Format market cap
        market_cap_formatted = f"${stock['market_cap'] / 1e12:.2f}T" if stock['market_cap'] >= 1e12 else f"${stock['market_cap'] / 1e9:.2f}B"
        
        # Determine if change is positive or negative
        change_indicator = "📈" if stock['change'] > 0 else "📉" if stock['change'] < 0 else "➡️"
        
        response = f"""
📊 STOCK INFORMATION: {symbol}

🏢 COMPANY: {stock['name']}
🏷️ SECTOR: {stock['sector']}

💰 PRICE INFORMATION:
Current Price: ${stock['current_price']:.2f}
Change: {change_indicator} ${stock['change']:+.2f} ({stock['change_percent']:+.2f}%)
Volume: {stock['volume']:,}

📈 KEY METRICS:
Market Cap: {market_cap_formatted}
P/E Ratio: {stock['pe_ratio']}
Dividend Yield: {stock['dividend_yield']}%

📊 52-WEEK RANGE:
High: ${stock['52_week_high']:.2f}
Low: ${stock['52_week_low']:.2f}

🎯 ANALYSIS:
The stock is trading at {((stock['current_price'] - stock['52_week_low']) / (stock['52_week_high'] - stock['52_week_low']) * 100):.1f}% of its 52-week range.
        """
        
        return response.strip()
        
    except Exception as e:
        return f"Error looking up stock information: {str(e)}"

# Create the stock lookup tool
stock_lookup_tool = StructuredTool.from_function(
    func=stock_price_lookup,
    name="StockPriceLookup",
    description="Look up current stock price and financial metrics for publicly traded companies. Use this tool to get detailed information about stocks including current price, daily change, volume, market cap, P/E ratio, and more. Provide the stock ticker symbol (e.g., AAPL, GOOGL, MSFT).",
    args_schema=StockLookupInput
)

# Test the tool
print("Testing Stock Price Lookup Tool:")
test_result = stock_lookup_tool.run({"symbol": "AAPL"})
print(test_result)
print("\n" + "="*50)
test_result2 = stock_lookup_tool.run({"symbol": "TSLA"})
print(test_result2)

Testing Stock Price Lookup Tool:
📊 STOCK INFORMATION: AAPL

🏢 COMPANY: Apple Inc.
🏷️ SECTOR: Technology

💰 PRICE INFORMATION:
Current Price: $185.92
Change: 📈 $+2.34 (+1.28%)
Volume: 52,840,000

📈 KEY METRICS:
Market Cap: $2.89T
P/E Ratio: 28.5
Dividend Yield: 0.44%

📊 52-WEEK RANGE:
High: $199.62
Low: $164.08

🎯 ANALYSIS:
The stock is trading at 61.5% of its 52-week range.

📊 STOCK INFORMATION: TSLA

🏢 COMPANY: Tesla Inc.
🏷️ SECTOR: Automotive

💰 PRICE INFORMATION:
Current Price: $248.50
Change: 📉 $-8.23 (-3.21%)
Volume: 98,650,000

📈 KEY METRICS:
Market Cap: $789.00B
P/E Ratio: 58.7
Dividend Yield: 0.0%

📊 52-WEEK RANGE:
High: $299.29
Low: $152.37

🎯 ANALYSIS:
The stock is trading at 65.4% of its 52-week range.


## Part 4: Portfolio Value Calculator Tool

Let's create one more tool that calculates the total value of a stock portfolio.

In [33]:
# Input schema for portfolio calculator
class PortfolioInput(BaseModel):
    """Input schema for portfolio value calculator."""
    holdings: str = Field(description="JSON string of stock holdings in format: '{\"AAPL\": 100, \"GOOGL\": 50, \"MSFT\": 25}'")

def portfolio_value_calculator(holdings: str) -> str:
    """
    Calculate the total value of a stock portfolio.
    
    Args:
        holdings: JSON string of stock holdings (symbol: number of shares)
    
    Returns:
        Portfolio value breakdown and total value
    """
    try:
        # Parse the holdings JSON
        holdings_dict = json.loads(holdings)
        
        portfolio_value = 0
        portfolio_breakdown = []
        total_change = 0
        
        for symbol, shares in holdings_dict.items():
            symbol = symbol.upper().strip()
            
            if symbol not in MOCK_STOCK_DATA:
                return f"❌ Symbol '{symbol}' not found in available data"
            
            stock = MOCK_STOCK_DATA[symbol]
            position_value = stock['current_price'] * shares
            position_change = stock['change'] * shares
            
            portfolio_value += position_value
            total_change += position_change
            
            portfolio_breakdown.append({
                "symbol": symbol,
                "name": stock['name'],
                "shares": shares,
                "price": stock['current_price'],
                "position_value": position_value,
                "position_change": position_change,
                "weight": 0  # Will calculate after we have total value
            })
        
        # Calculate position weights
        for position in portfolio_breakdown:
            position['weight'] = (position['position_value'] / portfolio_value) * 100
        
        # Build the response
        response = f"""
📊 PORTFOLIO ANALYSIS

💰 TOTAL PORTFOLIO VALUE: ${portfolio_value:,.2f}
📈 TOTAL DAILY CHANGE: ${total_change:+,.2f} ({(total_change/portfolio_value)*100:+.2f}%)

🏢 POSITION BREAKDOWN:
"""
        
        for position in sorted(portfolio_breakdown, key=lambda x: x['position_value'], reverse=True):
            change_indicator = "📈" if position['position_change'] > 0 else "📉" if position['position_change'] < 0 else "➡️"
            response += f"""
{position['symbol']} - {position['name']}
  • Shares: {position['shares']:,}
  • Price: ${position['price']:.2f}
  • Position Value: ${position['position_value']:,.2f} ({position['weight']:.1f}%)
  • Daily Change: {change_indicator} ${position['position_change']:+,.2f}
"""
        
        # Add diversification analysis
        response += f"""
🎯 DIVERSIFICATION ANALYSIS:
• Number of Holdings: {len(portfolio_breakdown)}
• Largest Position: {max(portfolio_breakdown, key=lambda x: x['weight'])['symbol']} ({max(portfolio_breakdown, key=lambda x: x['weight'])['weight']:.1f}%)
• Portfolio Concentration: {'High' if max(portfolio_breakdown, key=lambda x: x['weight'])['weight'] > 40 else 'Moderate' if max(portfolio_breakdown, key=lambda x: x['weight'])['weight'] > 20 else 'Low'}
"""
        
        return response.strip()
        
    except json.JSONDecodeError:
        return "❌ Invalid JSON format for holdings. Please provide holdings in format: '{\"AAPL\": 100, \"GOOGL\": 50}'"
    except Exception as e:
        return f"Error calculating portfolio value: {str(e)}"

# Create the portfolio calculator tool
portfolio_calculator_tool = StructuredTool.from_function(
    func=portfolio_value_calculator,
    name="PortfolioValueCalculator",
    description="Calculate the total value and breakdown of a stock portfolio. Use this tool to analyze portfolio value, daily changes, and diversification. Provide holdings as a JSON string with stock symbols and share quantities.",
    args_schema=PortfolioInput
)

# Test the tool
print("Testing Portfolio Value Calculator:")
test_portfolio = '{"AAPL": 100, "GOOGL": 50, "MSFT": 25, "TSLA": 10}'
test_result = portfolio_calculator_tool.run({"holdings": test_portfolio})
print(test_result)

Testing Portfolio Value Calculator:
📊 PORTFOLIO ANALYSIS

💰 TOTAL PORTFOLIO VALUE: $37,676.25
📈 TOTAL DAILY CHANGE: $+231.95 (+0.62%)

🏢 POSITION BREAKDOWN:

AAPL - Apple Inc.
  • Shares: 100
  • Price: $185.92
  • Position Value: $18,592.00 (49.3%)
  • Daily Change: 📈 $+234.00

MSFT - Microsoft Corporation
  • Shares: 25
  • Price: $378.85
  • Position Value: $9,471.25 (25.1%)
  • Daily Change: 📈 $+141.75

GOOGL - Alphabet Inc.
  • Shares: 50
  • Price: $142.56
  • Position Value: $7,128.00 (18.9%)
  • Daily Change: 📉 $-61.50

TSLA - Tesla Inc.
  • Shares: 10
  • Price: $248.50
  • Position Value: $2,485.00 (6.6%)
  • Daily Change: 📉 $-82.30

🎯 DIVERSIFICATION ANALYSIS:
• Number of Holdings: 4
• Largest Position: AAPL (49.3%)
• Portfolio Concentration: High


## Part 5: Creating an Agent with Custom Tools

Now let's create an agent that can use all our custom tools to provide comprehensive financial analysis.

In [35]:
# Create a list of all our tools
financial_tools = [
    calculator_tool,
    percentage_calculator,
    compound_interest_tool,
    stock_lookup_tool,
    portfolio_calculator_tool
]

# Create an agent with our custom tools
financial_agent = initialize_agent(
    tools=financial_tools,
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True
)

print("✓ Financial Agent created with custom tools!")
print(f"✓ Agent has access to {len(financial_tools)} tools:")
for tool in financial_tools:
    print(f"  - {tool.name}")

✓ Financial Agent created with custom tools!
✓ Agent has access to 5 tools:
  - Calculator
  - percentage_calculator
  - CompoundInterestCalculator
  - StockPriceLookup
  - PortfolioValueCalculator


## Part 6: Testing the Agent

Let's test our agent with various financial queries to see how it uses our custom tools.

In [36]:
# Test 1: Compound Interest Calculation
print("=== TEST 1: Compound Interest Calculation ===")
query1 = "If I invest $5,000 at 6% annual interest rate for 15 years with monthly compounding, how much will I have?"
response1 = financial_agent.run(query1)
print(f"Query: {query1}")
print(f"Response: {response1}")

print("\n" + "="*80 + "\n")

# Test 2: Stock Information
print("=== TEST 2: Stock Information ===")
query2 = "What's the current price and key metrics for Microsoft (MSFT)?"
response2 = financial_agent.run(query2)
print(f"Query: {query2}")
print(f"Response: {response2}")

print("\n" + "="*80 + "\n")

# Test 3: Portfolio Analysis
print("=== TEST 3: Portfolio Analysis ===")
query3 = "I have a portfolio with 75 shares of Apple, 30 shares of Google, and 20 shares of Tesla. What's my total portfolio value and how is it performing today?"
response3 = financial_agent.run(query3)
print(f"Query: {query3}")
print(f"Response: {response3}")

print("\n" + "="*80 + "\n")

# Test 4: Complex Financial Planning
print("=== TEST 4: Complex Financial Planning ===")
query4 = "I want to save $50,000 for a house down payment. If I can invest $500 monthly at 5% annual return, how long will it take? Also, what's Apple's current stock price?"
response4 = financial_agent.run(query4)
print(f"Query: {query4}")
print(f"Response: {response4}")

=== TEST 1: Compound Interest Calculation ===


> Entering new AgentExecutor chain...


/tmp/ipykernel_16365/1214437238.py:4: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response1 = financial_agent.run(query1)


Thought: I should use the CompoundInterestCalculator tool to calculate the future value of the investment with compound interest.
Action:
```
{
  "action": "CompoundInterestCalculator",
  "action_input": {
    "principal": 5000,
    "annual_rate": 0.06,
    "years": 15,
    "compounds_per_year": 12
  }
}
```
Observation: 📊 COMPOUND INTEREST CALCULATION RESULTS:

Initial Investment: $5,000.00
Annual Interest Rate: 6.0%
Investment Period: 15 years
Compounding Frequency: 12 times per year

💰 FINAL RESULTS:
Final Amount: $12,270.47
Interest Earned: $7,270.47
Total Return: 145.41%

📈 ADDITIONAL METRICS:
Effective Annual Rate: 6.17%
Average Monthly Growth: 0.5%

🎯 SUMMARY: Your $5,000.00 investment will grow to $12,270.47 over 15 years, earning $7,270.47 in compound interest!
Thought:I have calculated that if you invest $5,000 at a 6% annual interest rate for 15 years with monthly compounding, you will have $12,270.47. You will earn $7,270.47 in compound interest over the investment period.


## Part 7: Interactive Financial Assistant

Let's create an interactive function that allows users to easily interact with our financial agent.

In [38]:
def financial_assistant(query: str) -> str:
    """
    Interactive financial assistant that can answer questions using custom tools.
    
    Args:
        query: Financial question or request
    
    Returns:
        AI-powered response using financial tools
    """
    try:
        response = financial_agent.run(query)
        return response
    except Exception as e:
        return f"Sorry, I encountered an error: {str(e)}"

# Example usage
print("💡 FINANCIAL ASSISTANT EXAMPLES:")
print("="*50)

example_queries = [
    "Calculate compound interest on $10,000 at 8% for 20 years",
    "What's Tesla's current stock price?",
    "Analyze a portfolio with 50 AAPL, 25 GOOGL, 30 MSFT shares",
    "How much is 150 * 25 + 500?"
]

for i, query in enumerate(example_queries, 1):
    print(f"\n{i}. Query: {query}")
    print(f"   Answer: {financial_assistant(query)}")
    print("-" * 40)

print("\n🎯 Try your own queries using: financial_assistant('your question here')")

💡 FINANCIAL ASSISTANT EXAMPLES:

1. Query: Calculate compound interest on $10,000 at 8% for 20 years


> Entering new AgentExecutor chain...
Action:
```
{
  "action": "CompoundInterestCalculator",
  "action_input": {
    "principal": 10000,
    "annual_rate": 0.08,
    "years": 20,
    "compounds_per_year": 1
  }
}
```
Observation: 📊 COMPOUND INTEREST CALCULATION RESULTS:

Initial Investment: $10,000.00
Annual Interest Rate: 8.0%
Investment Period: 20 years
Compounding Frequency: 1 times per year

💰 FINAL RESULTS:
Final Amount: $46,609.57
Interest Earned: $36,609.57
Total Return: 366.1%

📈 ADDITIONAL METRICS:
Effective Annual Rate: 8.0%
Average Monthly Growth: 0.6434%

🎯 SUMMARY: Your $10,000.00 investment will grow to $46,609.57 over 20 years, earning $36,609.57 in compound interest!
Thought:Action:
```
{
  "action": "CompoundInterestCalculator",
  "action_input": {
    "principal": 10000,
    "annual_rate": 0.08,
    "years": 20,
    "compounds_per_year": 1
  }
}
```
Observation: 📊 C

## Summary & Best Practices

### What You've Learned 🎓

In this notebook, you've mastered creating custom tools for LangChain:

1. **Tool Creation**: Built tools using `Tool` class, `@tool` decorator, and `StructuredTool` classes
2. **Function Wrapping**: Converted Python functions into LangChain tools using multiple methods
3. **Decorator Usage**: Used the `@tool` decorator for clean, automatic tool creation
4. **Input Validation**: Used Pydantic models for structured tool inputs
5. **Financial Tools**: Created practical tools for financial calculations and data lookup
6. **Agent Integration**: Combined multiple tools into a comprehensive financial agent
7. **Error Handling**: Implemented robust error handling in tool functions

### Custom Tools Created 🛠️

- **Calculator Tool**: Basic mathematical operations using Tool() class
- **Percentage Calculator**: Percentage calculations using @tool decorator
- **Compound Interest Calculator**: Advanced financial calculations with detailed output
- **Stock Price Lookup**: Mock financial data retrieval with comprehensive metrics
- **Portfolio Value Calculator**: Multi-asset portfolio analysis and diversification insights

### Key Concepts Mastered 🔑

- **Tool Class**: Creating tools from functions with descriptions
- **@tool Decorator**: Simple, clean tool creation with automatic schema generation
- **StructuredTool**: Advanced tools with typed inputs and validation
- **Pydantic Schemas**: Defining input structures for tools
- **Agent Integration**: Using tools with LangChain agents
- **Error Handling**: Robust error management in tool functions
- **Tool Descriptions**: Writing clear descriptions for agent use

### Tool Creation Methods 🛠️

1. **Tool() Class Method**
   - Explicit control over all parameters
   - Good for simple functions
   - Manual schema definition

2. **@tool Decorator Method**
   - Clean, Pythonic syntax
   - Automatic schema generation from function signature
   - Best for straightforward tools

3. **StructuredTool.from_function()**
   - Advanced input validation with Pydantic
   - Complex parameter schemas
   - Best for sophisticated tools with multiple parameters

### Best Practices for Custom Tools 💡

1. **Clear Descriptions**
   - Write descriptive tool names and descriptions
   - Explain input parameters clearly
   - Include examples in descriptions

2. **Input Validation**
   - Use Pydantic models for structured inputs
   - Validate data types and ranges
   - Handle edge cases gracefully

3. **Error Handling**
   - Wrap tool functions in try-catch blocks
   - Return meaningful error messages
   - Don't let tools crash the agent

4. **Output Formatting**
   - Return well-formatted, readable responses
   - Include relevant context and explanations
   - Use consistent formatting across tools

5. **Testing**
   - Test tools independently before agent integration
   - Test with various input scenarios
   - Verify error handling works correctly

### Next Steps 🚀

- Create more specialized financial tools
- Add real API integrations for live data
- Build tools that work together for complex workflows
- Explore advanced agent architectures
- Move on to multi-agent systems!

Excellent work! You now have the skills to create powerful custom tools that extend LangChain's capabilities! 🎉